In [1]:
#Importing and loading libraries
import numpy as np
import pandas as pd
#For PCA
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
#For undersampling
from collections import Counter
from imblearn.under_sampling import NearMiss
#Train-test sets splitting
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_predict
#For DecisionTrees
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
#For Logistic Regression classifier
from sklearn.linear_model import LogisticRegression
#For Naive Bayes classifier
from sklearn.naive_bayes import GaussianNB
#For Support Vector Machine
from sklearn import svm
#For Random Forest
from sklearn.ensemble import RandomForestClassifier
#For visualisation
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv("C:\\Users\\akram\\Desktop\\Dissertation\\Datasets\\CompleteDatasetApproach2.csv")
#Removing irrelevant columns
df = df.drop(['PlayerName', 'Week'], axis = 1)
features = df.drop(['InjuredNextWeek'], axis = 1)
target = df['InjuredNextWeek']

In [3]:
#Looking at proportion of '0's and '1's in the target column
imbal_proportion = Counter(target)
print(imbal_proportion)
#Undersampling the training set
undersample = NearMiss(version=3, n_neighbors_ver3=3)
# transform the dataset
features_test, target_test = undersample.fit_resample(features, target)
#Looking at new proportion of '0's and '1's
bal_proportion = Counter(target_test)
print(bal_proportion)

Counter({0: 1158, 1: 29})
Counter({0: 29, 1: 29})


In [4]:
#Function to return accuracy metrics
def accuraciz(y_test, y_pred):
    count = Counter(y_test)
    real_values_zero = count[0]
    real_values_one = count[1]
    
    TP = 0
    FP = 0
    TN = 0
    FN = 0
    
    #Transform the data into a numerical vector
    y_test = list(map(int, y_test))
    y_pred = list(map(int, y_pred))
    
    for i in range(len(y_pred)):
        if y_test[i] == 1 and y_pred[i] == 1:
            TP += 1
        if y_pred[i] == 1 and y_test[i] != y_pred[i]:
            FP += 1
        if y_test[i] == y_pred[i] == 0:
            TN += 1
        if y_pred[i] == 0 and y_test[i] != y_pred[i]:
            FN += 1
    
    sensitivity = TP/(TP + FN)
    specificity = TN/(FP + TN)
    FPR = FP / real_values_zero
    FNR = FN / real_values_one
    acc = (TP + TN) / (real_values_one + real_values_zero)
    ba = (sensitivity + specificity) / 2
    f1 = (2*TP) / (2*TP + FP + FN)
    LRplus = sensitivity/(1-specificity)
    LRminus = (1-sensitivity)/specificity
    
    return sensitivity*100, specificity*100, FPR*100, FNR*100, acc*100, ba*100, f1*100, LRplus, LRminus, TP, FP, TN, FN

In [5]:
i = 1
sensitivities = []
specificities = []
FPRs = []
FNRs = []
ACCs = []
BAs = []
F1s = []
LRpluses = []
LRminuses = []
TPs = []
FPs = []
TNs = []
FNs = []

#Building the Decision Tree classifier
#Recursive to obtain mean and variance of results
#10-fold cross validation
myDT = DecisionTreeClassifier(criterion = 'gini', max_depth = 10)

while i < 101:
    features_kfold, target_kfold = undersample.fit_resample(features, target)
    y_pred_myDT = cross_val_predict(myDT, features_kfold, target_kfold, cv=10)

    performance = accuraciz(target_kfold, y_pred_myDT)
    sensitivities.append(performance[0])
    specificities.append(performance[1])
    FPRs.append(performance[2])
    FNRs.append(performance[3])
    ACCs.append(performance[4])
    BAs.append(performance[5])
    F1s.append(performance[6])
    LRpluses.append(performance[7])
    LRminuses.append(performance[8])
    TPs.append(performance[9])
    FPs.append(performance[10])
    TNs.append(performance[11])
    FNs.append(performance[12])
    
    i += 1
    
TPz = np.sum(TPs)
FPz = np.sum(FPs)
TNz = np.sum(TNs)
FNz = np.sum(FNs)
totz = TPz + FPz + TNz + FNz

print("TP: "+str(TPz)+" ("+str(TPz*100/totz)+"%), FP: "+str(FPz)+" ("+str(FPz*100/totz)+"%)")
print("TN: "+str(TNz)+" ("+str(TNz*100/totz)+"%), FN: "+str(FNz)+" ("+str(FNz*100/totz)+"%)")
print("Sensitivity has mean "+str(np.mean(sensitivities))+" with variance "+str(np.var(sensitivities)))
print("Specificity has mean "+str(np.mean(specificities))+" with variance "+str(np.var(specificities)))
print("False Positive Rate has mean "+str(np.mean(FPRs))+" with variance "+str(np.var(FPRs)))
print("False Negative Rate has mean "+str(np.mean(FNRs))+" with variance "+str(np.var(FNRs)))
print("ACC has mean "+str(np.mean(ACCs))+" with variance "+str(np.var(ACCs)))
print("Balanced Accuracy has mean "+str(np.mean(BAs))+" with variance "+str(np.var(BAs)))
print("F1-Score has mean "+str(np.mean(F1s))+" with variance "+str(np.var(F1s)))
print("Likelihood ratio positive has mean "+str(np.mean(LRpluses))+" with variance "+str(np.var(LRpluses)))
print("Likelihood ratio negative has mean "+str(np.mean(LRminuses))+" with variance "+str(np.var(LRminuses)))

TP: 1499 (25.844827586206897%), FP: 1622 (27.96551724137931%)
TN: 1278 (22.03448275862069%), FN: 1401 (24.155172413793103%)
Sensitivity has mean 51.68965517241379 with variance 12.008323424494641
Specificity has mean 44.068965517241395 with variance 6.321046373365042
False Positive Rate has mean 55.931034482758605 with variance 6.321046373365026
False Negative Rate has mean 48.31034482758621 with variance 12.008323424494645
ACC has mean 47.87931034482759 with variance 4.925980975029726
Balanced Accuracy has mean 47.87931034482759 with variance 4.925980975029722
F1-Score has mean 49.75961907530789 with variance 6.853465687757773
Likelihood ratio positive has mean 0.926281862745098 with variance 0.006004054756584012
Likelihood ratio negative has mean 1.1001373626373627 with variance 0.010951632502113264


In [6]:
i = 1
sensitivities = []
specificities = []
FPRs = []
FNRs = []
ACCs = []
BAs = []
F1s = []
LRpluses = []
LRminuses = []
TPs = []
FPs = []
TNs = []
FNs = []

#Building the Decision Tree classifier
#Recursive to obtain mean and variance of results
#10-fold cross validation
myDT = DecisionTreeClassifier(criterion = 'entropy', max_depth = 10)

while i < 101:
    features_kfold, target_kfold = undersample.fit_resample(features, target)
    y_pred_myDT = cross_val_predict(myDT, features_kfold, target_kfold, cv=10)

    performance = accuraciz(target_kfold, y_pred_myDT)
    sensitivities.append(performance[0])
    specificities.append(performance[1])
    FPRs.append(performance[2])
    FNRs.append(performance[3])
    ACCs.append(performance[4])
    BAs.append(performance[5])
    F1s.append(performance[6])
    LRpluses.append(performance[7])
    LRminuses.append(performance[8])
    TPs.append(performance[9])
    FPs.append(performance[10])
    TNs.append(performance[11])
    FNs.append(performance[12])
    
    i += 1
    
TPz = np.sum(TPs)
FPz = np.sum(FPs)
TNz = np.sum(TNs)
FNz = np.sum(FNs)
totz = TPz + FPz + TNz + FNz

print("TP: "+str(TPz)+" ("+str(TPz*100/totz)+"%), FP: "+str(FPz)+" ("+str(FPz*100/totz)+"%)")
print("TN: "+str(TNz)+" ("+str(TNz*100/totz)+"%), FN: "+str(FNz)+" ("+str(FNz*100/totz)+"%)")
print("Sensitivity has mean "+str(np.mean(sensitivities))+" with variance "+str(np.var(sensitivities)))
print("Specificity has mean "+str(np.mean(specificities))+" with variance "+str(np.var(specificities)))
print("False Positive Rate has mean "+str(np.mean(FPRs))+" with variance "+str(np.var(FPRs)))
print("False Negative Rate has mean "+str(np.mean(FNRs))+" with variance "+str(np.var(FNRs)))
print("ACC has mean "+str(np.mean(ACCs))+" with variance "+str(np.var(ACCs)))
print("Balanced Accuracy has mean "+str(np.mean(BAs))+" with variance "+str(np.var(BAs)))
print("F1-Score has mean "+str(np.mean(F1s))+" with variance "+str(np.var(F1s)))
print("Likelihood ratio positive has mean "+str(np.mean(LRpluses))+" with variance "+str(np.var(LRpluses)))
print("Likelihood ratio negative has mean "+str(np.mean(LRminuses))+" with variance "+str(np.var(LRminuses)))

TP: 1619 (27.913793103448278%), FP: 1508 (26.0%)
TN: 1392 (24.0%), FN: 1281 (22.086206896551722%)
Sensitivity has mean 55.82758620689654 with variance 11.104637336504146
Specificity has mean 48.0 with variance 16.80856123662306
False Positive Rate has mean 52.0 with variance 16.808561236623053
False Negative Rate has mean 44.17241379310346 with variance 11.104637336504162
ACC has mean 51.91379310344828 with variance 8.376634958382875
Balanced Accuracy has mean 51.91379310344828 with variance 8.376634958382871
F1-Score has mean 53.71391631849474 with variance 7.646100530027121
Likelihood ratio positive has mean 1.0813071715865832 with variance 0.013584659662607157
Likelihood ratio negative has mean 0.9285117872323757 with variance 0.014193384881158082


In [7]:
i = 1
sensitivities = []
specificities = []
FPRs = []
FNRs = []
ACCs = []
BAs = []
F1s = []
LRpluses = []
LRminuses = []
TPs = []
FPs = []
TNs = []
FNs = []

#Building the Decision Tree classifier
#Recursive to obtain mean and variance of results
#10-fold cross validation
myCLR = LogisticRegression(solver='lbfgs', max_iter=1000)

while i < 101:
    features_kfold, target_kfold = undersample.fit_resample(features, target)
    y_pred_myCLR = cross_val_predict(myCLR, features_kfold, target_kfold, cv=10)

    performance = accuraciz(target_kfold, y_pred_myCLR)
    sensitivities.append(performance[0])
    specificities.append(performance[1])
    FPRs.append(performance[2])
    FNRs.append(performance[3])
    ACCs.append(performance[4])
    BAs.append(performance[5])
    F1s.append(performance[6])
    LRpluses.append(performance[7])
    LRminuses.append(performance[8])
    TPs.append(performance[9])
    FPs.append(performance[10])
    TNs.append(performance[11])
    FNs.append(performance[12])
    
    i += 1
    
TPz = np.sum(TPs)
FPz = np.sum(FPs)
TNz = np.sum(TNs)
FNz = np.sum(FNs)
totz = TPz + FPz + TNz + FNz

print("TP: "+str(TPz)+" ("+str(TPz*100/totz)+"%), FP: "+str(FPz)+" ("+str(FPz*100/totz)+"%)")
print("TN: "+str(TNz)+" ("+str(TNz*100/totz)+"%), FN: "+str(FNz)+" ("+str(FNz*100/totz)+"%)")
print("Sensitivity has mean "+str(np.mean(sensitivities))+" with variance "+str(np.var(sensitivities)))
print("Specificity has mean "+str(np.mean(specificities))+" with variance "+str(np.var(specificities)))
print("False Positive Rate has mean "+str(np.mean(FPRs))+" with variance "+str(np.var(FPRs)))
print("False Negative Rate has mean "+str(np.mean(FNRs))+" with variance "+str(np.var(FNRs)))
print("ACC has mean "+str(np.mean(ACCs))+" with variance "+str(np.var(ACCs)))
print("Balanced Accuracy has mean "+str(np.mean(BAs))+" with variance "+str(np.var(BAs)))
print("F1-Score has mean "+str(np.mean(F1s))+" with variance "+str(np.var(F1s)))
print("Likelihood ratio positive has mean "+str(np.mean(LRpluses))+" with variance "+str(np.var(LRpluses)))
print("Likelihood ratio negative has mean "+str(np.mean(LRminuses))+" with variance "+str(np.var(LRminuses)))

C:\Users\akram\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\akram\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/m

C:\Users\akram\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\akram\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/m

C:\Users\akram\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\akram\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/m

C:\Users\akram\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\akram\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/m

C:\Users\akram\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\akram\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/m

...

C:\Users\akram\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\akram\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/m

C:\Users\akram\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\akram\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/m

C:\Users\akram\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\akram\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/m

TP: 1200 (20.689655172413794%), FP: 1700 (29.310344827586206%)
TN: 1200 (20.689655172413794%), FN: 1700 (29.310344827586206%)
Sensitivity has mean 41.37931034482758 with variance 5.048709793414476e-29
Specificity has mean 41.37931034482758 with variance 5.048709793414476e-29
False Positive Rate has mean 58.6206896551724 with variance 5.048709793414476e-29
False Negative Rate has mean 58.6206896551724 with variance 5.048709793414476e-29
ACC has mean 41.37931034482758 with variance 5.048709793414476e-29
Balanced Accuracy has mean 41.37931034482758 with variance 5.048709793414476e-29
F1-Score has mean 41.37931034482758 with variance 5.048709793414476e-29
Likelihood ratio positive has mean 0.7058823529411764 with variance 0.0
Likelihood ratio negative has mean 1.416666666666666 with variance 4.437342591868191e-31


C:\Users\akram\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\akram\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/m

In [8]:
i = 1
sensitivities = []
specificities = []
FPRs = []
FNRs = []
ACCs = []
BAs = []
F1s = []
LRpluses = []
LRminuses = []
TPs = []
FPs = []
TNs = []
FNs = []

#Building the Naive-Bayes classifier
#Recursive to obtain mean and variance of results
#10-fold cross validation
myNB = GaussianNB()

while i < 101:
    features_kfold, target_kfold = undersample.fit_resample(features, target)
    y_pred_myNB = cross_val_predict(myNB, features_kfold, target_kfold, cv=10)

    performance = accuraciz(target_kfold, y_pred_myNB)
    sensitivities.append(performance[0])
    specificities.append(performance[1])
    FPRs.append(performance[2])
    FNRs.append(performance[3])
    ACCs.append(performance[4])
    BAs.append(performance[5])
    F1s.append(performance[6])
    LRpluses.append(performance[7])
    LRminuses.append(performance[8])
    TPs.append(performance[9])
    FPs.append(performance[10])
    TNs.append(performance[11])
    FNs.append(performance[12])
    
    i += 1

TPz = np.sum(TPs)
FPz = np.sum(FPs)
TNz = np.sum(TNs)
FNz = np.sum(FNs)
totz = TPz + FPz + TNz + FNz

print("TP: "+str(TPz)+" ("+str(TPz*100/totz)+"%), FP: "+str(FPz)+" ("+str(FPz*100/totz)+"%)")
print("TN: "+str(TNz)+" ("+str(TNz*100/totz)+"%), FN: "+str(FNz)+" ("+str(FNz*100/totz)+"%)")
print("Sensitivity has mean "+str(np.mean(sensitivities))+" with variance "+str(np.var(sensitivities)))
print("Specificity has mean "+str(np.mean(specificities))+" with variance "+str(np.var(specificities)))
print("False Positive Rate has mean "+str(np.mean(FPRs))+" with variance "+str(np.var(FPRs)))
print("False Negative Rate has mean "+str(np.mean(FNRs))+" with variance "+str(np.var(FNRs)))
print("ACC has mean "+str(np.mean(ACCs))+" with variance "+str(np.var(ACCs)))
print("Balanced Accuracy has mean "+str(np.mean(BAs))+" with variance "+str(np.var(BAs)))
print("F1-Score has mean "+str(np.mean(F1s))+" with variance "+str(np.var(F1s)))
print("Likelihood ratio positive has mean "+str(np.mean(LRpluses))+" with variance "+str(np.var(LRpluses)))
print("Likelihood ratio negative has mean "+str(np.mean(LRminuses))+" with variance "+str(np.var(LRminuses)))

TP: 1600 (27.586206896551722%), FP: 1500 (25.862068965517242%)
TN: 1400 (24.137931034482758%), FN: 1300 (22.413793103448278%)
Sensitivity has mean 55.172413793103445 with variance 0.0
Specificity has mean 48.275862068965516 with variance 0.0
False Positive Rate has mean 51.72413793103449 with variance 5.048709793414476e-29
False Negative Rate has mean 44.82758620689657 with variance 2.0194839173657902e-28
ACC has mean 51.72413793103449 with variance 5.048709793414476e-29
Balanced Accuracy has mean 51.72413793103449 with variance 5.048709793414476e-29
F1-Score has mean 53.33333333333332 with variance 2.0194839173657902e-28
Likelihood ratio positive has mean 1.0666666666666664 with variance 1.9721522630525295e-31
Likelihood ratio negative has mean 0.9285714285714287 with variance 4.930380657631324e-32


In [9]:
i = 1
sensitivities = []
specificities = []
FPRs = []
FNRs = []
ACCs = []
BAs = []
F1s = []
LRpluses = []
LRminuses = []
TPs = []
FPs = []
TNs = []
FNs = []

#Building the Support Vector classifier
#Recursive to obtain mean and variance of results
#10-fold cross validation
mySVC = svm.SVC()

while i < 101:
    features_kfold, target_kfold = undersample.fit_resample(features, target)
    y_pred_mySVC = cross_val_predict(mySVC, features_kfold, target_kfold, cv=10)

    performance = accuraciz(target_kfold, y_pred_mySVC)
    sensitivities.append(performance[0])
    specificities.append(performance[1])
    FPRs.append(performance[2])
    FNRs.append(performance[3])
    ACCs.append(performance[4])
    BAs.append(performance[5])
    F1s.append(performance[6])
    LRpluses.append(performance[7])
    LRminuses.append(performance[8])
    TPs.append(performance[9])
    FPs.append(performance[10])
    TNs.append(performance[11])
    FNs.append(performance[12])
    
    i += 1

TPz = np.sum(TPs)
FPz = np.sum(FPs)
TNz = np.sum(TNs)
FNz = np.sum(FNs)
totz = TPz + FPz + TNz + FNz

print("TP: "+str(TPz)+" ("+str(TPz*100/totz)+"%), FP: "+str(FPz)+" ("+str(FPz*100/totz)+"%)")
print("TN: "+str(TNz)+" ("+str(TNz*100/totz)+"%), FN: "+str(FNz)+" ("+str(FNz*100/totz)+"%)")
print("Sensitivity has mean "+str(np.mean(sensitivities))+" with variance "+str(np.var(sensitivities)))
print("Specificity has mean "+str(np.mean(specificities))+" with variance "+str(np.var(specificities)))
print("False Positive Rate has mean "+str(np.mean(FPRs))+" with variance "+str(np.var(FPRs)))
print("False Negative Rate has mean "+str(np.mean(FNRs))+" with variance "+str(np.var(FNRs)))
print("ACC has mean "+str(np.mean(ACCs))+" with variance "+str(np.var(ACCs)))
print("Balanced Accuracy has mean "+str(np.mean(BAs))+" with variance "+str(np.var(BAs)))
print("F1-Score has mean "+str(np.mean(F1s))+" with variance "+str(np.var(F1s)))
print("Likelihood ratio positive has mean "+str(np.mean(LRpluses))+" with variance "+str(np.var(LRpluses)))
print("Likelihood ratio negative has mean "+str(np.mean(LRminuses))+" with variance "+str(np.var(LRminuses)))

TP: 2100 (36.206896551724135%), FP: 1900 (32.758620689655174%)
TN: 1000 (17.24137931034483%), FN: 800 (13.793103448275861%)
Sensitivity has mean 72.41379310344827 with variance 0.0
Specificity has mean 34.48275862068966 with variance 0.0
False Positive Rate has mean 65.51724137931036 with variance 2.0194839173657902e-28
False Negative Rate has mean 27.586206896551722 with variance 0.0
ACC has mean 53.44827586206896 with variance 0.0
Balanced Accuracy has mean 53.44827586206896 with variance 2.0194839173657902e-28
F1-Score has mean 60.86956521739128 with variance 8.077935669463161e-28
Likelihood ratio positive has mean 1.105263157894737 with variance 0.0
Likelihood ratio negative has mean 0.7999999999999998 with variance 1.232595164407831e-32


In [10]:
i = 1
sensitivities = []
specificities = []
FPRs = []
FNRs = []
ACCs = []
BAs = []
F1s = []
LRpluses = []
LRminuses = []
TPs = []
FPs = []
TNs = []
FNs = []

#Building the Support Vector classifier
#Recursive to obtain mean and variance of results
#10-fold cross validation
myRF = RandomForestClassifier(criterion = 'entropy', max_depth = 10)

while i < 101:
    features_kfold, target_kfold = undersample.fit_resample(features, target)
    y_pred_myRF = cross_val_predict(myRF, features_kfold, target_kfold, cv=10)

    performance = accuraciz(target_kfold, y_pred_myRF)
    sensitivities.append(performance[0])
    specificities.append(performance[1])
    FPRs.append(performance[2])
    FNRs.append(performance[3])
    ACCs.append(performance[4])
    BAs.append(performance[5])
    F1s.append(performance[6])
    LRpluses.append(performance[7])
    LRminuses.append(performance[8])
    TPs.append(performance[9])
    FPs.append(performance[10])
    TNs.append(performance[11])
    FNs.append(performance[12])
    
    i += 1

TPz = np.sum(TPs)
FPz = np.sum(FPs)
TNz = np.sum(TNs)
FNz = np.sum(FNs)
totz = TPz + FPz + TNz + FNz

print("TP: "+str(TPz)+" ("+str(TPz*100/totz)+"%), FP: "+str(FPz)+" ("+str(FPz*100/totz)+"%)")
print("TN: "+str(TNz)+" ("+str(TNz*100/totz)+"%), FN: "+str(FNz)+" ("+str(FNz*100/totz)+"%)")
print("Sensitivity has mean "+str(np.mean(sensitivities))+" with variance "+str(np.var(sensitivities)))
print("Specificity has mean "+str(np.mean(specificities))+" with variance "+str(np.var(specificities)))
print("False Positive Rate has mean "+str(np.mean(FPRs))+" with variance "+str(np.var(FPRs)))
print("False Negative Rate has mean "+str(np.mean(FNRs))+" with variance "+str(np.var(FNRs)))
print("ACC has mean "+str(np.mean(ACCs))+" with variance "+str(np.var(ACCs)))
print("Balanced Accuracy has mean "+str(np.mean(BAs))+" with variance "+str(np.var(BAs)))
print("F1-Score has mean "+str(np.mean(F1s))+" with variance "+str(np.var(F1s)))
print("Likelihood ratio positive has mean "+str(np.mean(LRpluses))+" with variance "+str(np.var(LRpluses)))
print("Likelihood ratio negative has mean "+str(np.mean(LRminuses))+" with variance "+str(np.var(LRminuses)))

TP: 1398 (24.103448275862068%), FP: 1504 (25.93103448275862%)
TN: 1396 (24.06896551724138%), FN: 1502 (25.896551724137932%)
Sensitivity has mean 48.20689655172414 with variance 14.02615933412603
Specificity has mean 48.137931034482754 with variance 23.524375743162885
False Positive Rate has mean 51.862068965517246 with variance 23.524375743162878
False Negative Rate has mean 51.79310344827587 with variance 14.02615933412603
ACC has mean 48.17241379310345 with variance 7.7181926278240205
Balanced Accuracy has mean 48.17241379310345 with variance 7.7181926278240205
F1-Score has mean 48.157945849772815 with variance 8.124394194086989
Likelihood ratio positive has mean 0.9366868221647634 with variance 0.011077185159079541
Likelihood ratio negative has mean 1.0854743344890403 with variance 0.015265592641648548
